In [6]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

In [7]:
df_order_historic_demand = pd.read_excel('../../Datos_P1/df_historic_order_demand.xlsx')
df_distances = pd.read_excel('../../Datos_P1/df_distance_km.xlsx')
df_distance_min = pd.read_excel('../../Datos_P1/df_distance_min.xlsx')
df_location = pd.read_excel('../../Datos_P1/df_location.xlsx')
df_customers = pd.read_excel('../../Datos_P1/df_orders.xlsx')
df_vehicles = pd.read_excel('../../Datos_P1/df_vehicle.xlsx')

In [8]:
#cambiar posicion de almacen por mas comodidad

# Mostrar las primeras filas para verificar el contenido original
print("Datos originales:")
print(df_distances.head())


Datos originales:
   Cliente_1  Cliente_2  Cliente_3  Cliente_4  Cliente_5  Cliente_6  \
0     0.0000     7.5625    15.5365     1.1998     4.7145     1.7407   
1     7.5625     0.0000     3.3838     7.7433    14.5720     8.5237   
2    15.5365     3.3838     0.0000    12.5438     0.0000     0.0000   
3     1.1998     7.7433    12.5438     0.0000     5.0721     0.9119   
4     4.7145    14.5720     0.0000     5.0721     0.0000     4.8187   

   Cliente_7  Cliente_8  Cliente_9  Cliente_10  ...  Cliente_12  Cliente_13  \
0     7.9408    17.1947     4.2933      3.2659  ...      6.0225      5.4470   
1     0.4847    13.7974    10.1522      7.1521  ...     10.1049      2.6961   
2     0.0000    16.0355    13.9120     13.0649  ...     12.3430      5.0114   
3     7.5798    17.4095     3.5781      3.3451  ...      6.2330      4.7117   
4     0.0000     0.0000     0.0000      7.2170  ...      6.8738      9.1064   

   Cliente_14  Cliente_15  Cliente_16  Cliente_17  Cliente_18  Cliente_19  \
0  

In [9]:
def calculo_Probabilidad(current_node, unvisited_nodes, pheromones, df_distances):
    #Estas dos variables imprescindibles para ACO
    alpha = 1.0
    beta = 2.0
    probabilities = []
    for node in unvisited_nodes:
        distance = df_distances.iloc[current_node, node]
        if distance > 0:  
            tau = pheromones[current_node][node] ** alpha
            eta = (1 / distance) ** beta
            probabilities.append(tau * eta)
        else:
            probabilities.append(0)  
    
    probabilities = np.array(probabilities)
    total = probabilities.sum()
    
    # Evitar misma posicion entre clientes 
    if total == 0:
        probabilities = np.ones(len(unvisited_nodes)) / len(unvisited_nodes)
    else:
        probabilities = probabilities / total
    
    return probabilities

In [10]:
def calculo_Coste(solution, df_distances, df_vehicles):
    total_cost = 0
    vehicle_costes = []
    for vehicle_name, routes in solution:
        vehicle = df_vehicles.loc[df_vehicles["vehiculo_id"] == vehicle_name]
        price_per_km = vehicle["costo_km"].values[0]
        vehicle_total = 0
        
        for route in routes:
            route_cost = sum(df_distances.iloc[route[i], route[i + 1]] for i in range(len(route) - 1))
            vehicle_total += route_cost * price_per_km
        total_cost += vehicle_total
        vehicle_costes.append((vehicle_name, vehicle_total))
    return total_cost, vehicle_costes

In [11]:
def solucion_Vehiculo(df_distances, df_vehicles_shuffled, orders,pheromone_matrix, autonomia_restante=None):
    solution = []
    remaining_customers = set(range(20))
    depot = 20

    if autonomia_restante is None:
        autonomia_restante = {row["vehiculo_id"]: row["autonomia_km"] for _, row in df_vehicles_shuffled.iterrows()}

    for _, vehicle in df_vehicles_shuffled.iterrows():  # Usar el DataFrame reordenado
        vehicle_id = vehicle["vehiculo_id"]
        vehicle_routes = []
        vehicle_capacity = vehicle["capacidad_kg"]
        current_autonomy = autonomia_restante[vehicle_id]

        while remaining_customers and current_autonomy > 0:
            current_route = [depot]  # Inicia en el almacén
            current_capacity = vehicle_capacity
            current_node = depot
            total_distance = 0

            while remaining_customers:
                unvisited_nodes = [
                    node for node in remaining_customers
                    if orders[node] <= current_capacity 
                    and (total_distance + df_distances.iloc[current_node, node] + df_distances.iloc[node, depot]) <= current_autonomy
                ]

                if not unvisited_nodes:
                    break

                probabilities = calculo_Probabilidad(current_node, unvisited_nodes, pheromone_matrix, df_distances)
                next_node = np.random.choice(unvisited_nodes, p=probabilities)

                distance_to_next = df_distances.iloc[current_node, next_node]
                total_distance += distance_to_next
                current_autonomy -= distance_to_next

                current_route.append(next_node)
                current_capacity -= orders[next_node]
                remaining_customers.remove(next_node)
                current_node = next_node

            # Verificar si se agregaron clientes a la ruta
            if len(current_route) == 1:  # Solo el almacén, no hay clientes
                break  # Salir del bucle externo para este vehículo
            else:
                # Regresar al almacén y actualizar autonomía
                distance_to_depot = df_distances.iloc[current_node, depot]
                current_autonomy -= distance_to_depot
                current_route.append(depot)
                if len(current_route) > 2:
                    vehicle_routes.append(current_route)

            # Actualizar autonomía restante después de cada ruta
            autonomia_restante[vehicle_id] = current_autonomy

        if vehicle_routes:
            solution.append((vehicle_id, vehicle_routes))

    return solution, autonomia_restante




In [12]:
# Parámetros de ACO
num_vehiculos = 10
num_iterations = 100
rho = 0.5
Q = 100

n_nodes = len(df_customers)+1
pheromone_matrix = np.ones((n_nodes, n_nodes))
n_nodes = len(df_customers)+1
pheromone_matrix = np.ones((n_nodes, n_nodes))

solucion_optima = None
mejor_precio = float('inf')

for iteration in range(num_iterations):
    # Reordenar aleatoriamente los vehículos en cada iteración
    df_vehicles_shuffled = df_vehicles.sample(frac=1).reset_index(drop=True)

    soluciones = []
    costes = []

    for _ in range(num_vehiculos):
        solution, _ = solucion_Vehiculo(df_distances, df_vehicles_shuffled, df_customers["order_demand"],pheromone_matrix, None)
        total_cost, _ = calculo_Coste(solution, df_distances, df_vehicles_shuffled)
        soluciones.append(solution)
        costes.append(total_cost)

        if total_cost < mejor_precio:
            mejor_precio = total_cost
            solucion_optima = solution

    # Actualizar feromonas
    pheromone_matrix *= (1 - rho)
    for solution, cost in zip(soluciones, costes):
        for vehicle_name, routes in solution:
            for route in routes:
                for i in range(len(route) - 1):
                    pheromone_matrix[route[i]][route[i + 1]] += Q / cost

print(solucion_optima, mejor_precio)

[(2.0, [[20, 8, 5, 3, 0, 20], [20, 10, 13, 4, 16, 20], [20, 15, 6, 1, 12, 20], [20, 9, 11, 14, 7, 20], [20, 19, 2, 18, 17, 20]])] 22.69883


In [15]:
#Este es exactamente igual a la celda anterior pero para demostrar lka evoluciñon de las iteraciones con sus rutas
# Output
for iteration in range(num_iterations):
    print(f"\n🔄 Iteración {iteration + 1} 🔄")

    # Reordenar aleatoriamente los vehículos en cada iteración
    df_vehicles_shuffled = df_vehicles.sample(frac=1).reset_index(drop=True)

    soluciones = []
    costes = []

    for vehiculo_idx in range(num_vehiculos):
        solution, _ = solucion_Vehiculo(df_distances, df_vehicles_shuffled, df_customers["order_demand"], pheromone_matrix, None)
        total_cost, _ = calculo_Coste(solution, df_distances, df_vehicles_shuffled)
        soluciones.append(solution)
        costes.append(total_cost)

        # Imprimir detalle de la solución del vehículo
        print(f"\n🚚 Ruta {vehiculo_idx + 1}:")
        for vehicle_id, routes in solution:
            print(f"  Vehículo {vehicle_id} - Rutas: {routes}")
        print(f"  Costo total: {total_cost:.2f}")

        # Actualizar mejor solución
        if total_cost < mejor_precio:
            mejor_precio = total_cost
            solucion_optima = solution

    # Actualizar feromonas
    pheromone_matrix *= (1 - rho)
    for solution, cost in zip(soluciones, costes):
        for vehicle_name, routes in solution:
            for route in routes:
                for i in range(len(route) - 1):
                    pheromone_matrix[route[i]][route[i + 1]] += Q / cost

# Imprimir mejor solución encontrada
print("\n✅ Mejor solución encontrada:")
print(solucion_optima)
print(f"💰 Mejor precio: {mejor_precio:.2f}")


🔄 Iteración 1 🔄

🚚 Ruta 1:
  Vehículo 6.0 - Rutas: [[20, 8, 5, 3, 20], [20, 6, 1, 12, 20], [20, 10, 13, 4, 20], [20, 0, 15, 2, 20], [20, 9, 17, 14, 20], [20, 16, 11, 18, 20], [20, 19, 7, 20]]
  Costo total: 32.62

🚚 Ruta 2:
  Vehículo 6.0 - Rutas: [[20, 9, 17, 14, 20], [20, 8, 5, 3, 20], [20, 10, 13, 4, 20], [20, 16, 11, 19, 20], [20, 6, 1, 12, 20], [20, 0, 15, 2, 20], [20, 7, 18, 20]]
  Costo total: 30.87

🚚 Ruta 3:
  Vehículo 6.0 - Rutas: [[20, 8, 5, 3, 20], [20, 10, 13, 4, 20], [20, 0, 15, 12, 20], [20, 9, 17, 14, 20], [20, 16, 11, 19, 20], [20, 6, 1, 2, 20], [20, 7, 18, 20]]
  Costo total: 30.21

🚚 Ruta 4:
  Vehículo 6.0 - Rutas: [[20, 8, 5, 3, 20], [20, 9, 11, 14, 20], [20, 10, 13, 4, 20], [20, 15, 0, 6, 20], [20, 2, 17, 19, 20], [20, 12, 1, 18, 20], [20, 16, 7, 20]]
  Costo total: 32.63

🚚 Ruta 5:
  Vehículo 6.0 - Rutas: [[20, 8, 5, 3, 20], [20, 10, 13, 4, 20], [20, 9, 11, 6, 20], [20, 12, 1, 2, 20], [20, 15, 0, 16, 20], [20, 19, 7, 14, 20], [20, 18, 17, 20]]
  Costo total: 31.3